## Clean up Data into X and y arrays

In [1]:
import csv 

with open("/Users/alexraman/Downloads/TrainData.csv") as f:
    reader = csv.reader(f)
    header_names = next(reader)
    data = []
    for row in reader:
        data.append(row)
        
print(data[0])
print(type(data[0]))


['0.008293249295', '0.000095877277', '4.5', '4', '4.50', '4.52467232074017', '0.653652515029939', '0.015805705474', '0.017347725520', '0.055127216653', '0.249807247494', '0.661912104857', '0', '5', '-0.0714285714285714', '4.21428571428571', '0.000000000000', '1.000000000000', '1.000000000000', '0.000000000000', '0.000000000000', '1.000000000000', '1.000000000000', '0.000000000000', '5']
<class 'list'>


In [2]:
print(len(data))

150197


In [3]:
print(data[0][0:len(data[0])-1])

X = [elem[0:len(elem)-1] for i, elem in enumerate(data) if i < len(data)-1]
y = [elem[-1] for i, elem in enumerate(data) if i < len(data)-1]


print(X[-1])
print(y[-1])
print(header_names)
    

['0.008293249295', '0.000095877277', '4.5', '4', '4.50', '4.52467232074017', '0.653652515029939', '0.015805705474', '0.017347725520', '0.055127216653', '0.249807247494', '0.661912104857', '0', '5', '-0.0714285714285714', '4.21428571428571', '0.000000000000', '1.000000000000', '1.000000000000', '0.000000000000', '0.000000000000', '1.000000000000', '1.000000000000', '0.000000000000']
['0.660806103831', '0.036361457334', '4.02', '4', '4.00', '4.12082281301054', '0.821717139489925', '0.018342492202', '0.040843606119', '0.125129956928', '0.433016485964', '0.382667458785', '-8.11767379939605E-05', '3.84663008301674', '0.000332617500657355', '4.09031104298397', '0.328502415458', '0.671497584541', '0.386473429951', '0.285024154589', '0.022801302931', '0.977198697068', '0.654723127035', '0.322475570032']
4
['\ufeffBusinessReviewCount', 'UserReviewCount', 'RealUserAverageStars', 'BusinessAverageStars', 'NearestUserAverageStars', 'ReviewCategoryAverageStars', 'ReviewCategoryVariance', 'OneStar', 

In [4]:
for i, row in enumerate(X):
    for j, num in enumerate(row):
        X[i][j] = float(num)
        
for k, num in enumerate(y):
    y[k] = float(num)
    
print(X[0])
print(type(y[0]))

[0.008293249295, 9.5877277e-05, 4.5, 4.0, 4.5, 4.52467232074017, 0.653652515029939, 0.015805705474, 0.01734772552, 0.055127216653, 0.249807247494, 0.661912104857, 0.0, 5.0, -0.0714285714285714, 4.21428571428571, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0]
<class 'float'>


## Throw Data into SVM

In [5]:
import numpy as np

X_np = np.array(X)
y_np = np.array(y)


from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, test_size=0.33, random_state=42)

clf = SVC(kernel = 'linear')
clf.fit(X_train, y_train) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [6]:
y_pred = clf.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.5122566327045294

In [7]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[ 1005,    34,   699,   936,   235],
       [  439,    47,  1492,  2112,   356],
       [  157,    33,  2522,  5744,   807],
       [  114,    15,  1305, 13012,  3247],
       [   98,     4,   312,  6036,  8804]])

In [19]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

1.0643801069302936

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC


tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

clf_2 = GridSearchCV(SVC(), tuned_parameters, cv=5)
clf_2.fit(X_train, y_train)

print("Best parameters set found on development set:")
print()
print(clf_2.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf_2.cv_results_['mean_test_score']
stds = clf_2.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf_2.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, clf_2.predict(X_test)
print(classification_report(y_true, y_pred))
print()

